In [1]:
!ls

Desktop    Downloads  localperl  Pictures  snap       Videos
Documents  gopath     Music	 Public    Templates


In [2]:
%cd ../../media/hossein/HDD/gdrive/Active_projects/
!ls

/media/hossein/HDD/gdrive/Active_projects
 Archived_projects					     Data
'battery SoC estimation  Kalman Electrochemical model.pdf'   DEMOsDERAMAS
 BESTIES						     DISMAL
 BoarL							     eco-sim
 Collaborations						     my_dfs
 D3RM2							     ToDonextProjects


In [3]:
%cd my_dfs/
%mkdir protobuf
%cd protobuf

/media/hossein/HDD/gdrive/Active_projects/my_dfs
/media/hossein/HDD/gdrive/Active_projects/my_dfs/protobuf


Okay So here we want to run a simple example using the google protobuf ( aka gRPC) and using python.

Well I start from this guide

https://developers.google.com/protocol-buffers/docs/pythontutorial


Okay so not going too far in the tutorial, we see that we would need to install the complier for the protobuf


So in order to install the complier we need to get it from here:

https://github.com/protocolbuffers/protobuf/releases/tag/v3.13.0

It says you would follow the simple instructions for installation...



## Setup protobuf for python in docker container



So since this process of compiler setup could get messy and also since it would be good to have a clean isolated environment, why not have it setup in a container?!

okay, we start by a simple ubunto image with python and pip, etc.
(oh actually start with a repo(ish) and a Dockerfile

```
FROM ubuntu

RUN apt-get update -y 
RUN apt-get install -y git wget build-essential

RUN apt-get update && apt-get -y install python3 
```

Then it comes to installing the protobuf compiler.

>If you haven't installed the compiler, download the package from ***here*** and follow the instructions in the README.

so I go to **here** and I see this:

>> The latest release of Protocol Buffers can be found on the release page.


so then I go to the release page and download this package:

[  protobuf-all-3.13.0.tar.gz
](https://github.com/protocolbuffers/protobuf/releases/download/v3.13.0/protobuf-all-3.13.0.tar.gz)

on my pc to see what is going on. I check the Readme and find this 


> This README file contains protobuf installation instructions. To install
protobuf, you need to install the protocol compiler (used to compile .proto
files) and the protobuf runtime for your chosen programming language.





### Installing the compiler 


okay so now I know that the set up has two steps.  the complier and Runtime for python (which I don't know what that is)

okay it says:

> For non-C++ users, the simplest way to install the protocol compiler is to
download a pre-built binary from our release page:

>  [https://github.com/protocolbuffers/protobuf/releases](https://github.com/protocolbuffers/protobuf/releases)

> In the downloads section of each release, you can find pre-built binaries in
zip packages: protoc-$VERSION-$PLATFORM.zip. It contains the protoc binary
as well as a set of standard .proto files distributed along with protobuf.

okay so I gussed I need to only download these binaries and I will be good.
I check the release page and I see thes two
>protobuf-python-3.13.0.tar.gz

>protobuf-python-3.13.0.zip

I think that well the tar file would be better suited for my linux container.

so I use this 

```
RUN wget https://github.com/protocolbuffers/protobuf/releases/download/v3.13.0/protobuf-python-3.13.0.tar.gz -P /protobuf/source

RUN  tar -zxvf /protobuf/source/protobuf-python-3.13.0.tar.gz
```

at the same time I look at the archive of tar for python and the initial one with **all** label.

Then I thought the content of the tar fie for all and the zip file for python is essentially very different ( the all version also had a python folder, ( well at the time I figured that the python version would be essentially what is inside the python directory in the all version whihc was not the case!) 

so I thought the guide says get the zip file, maybe the zip file and the tar files are essentially different things (pretty odd huh?! well now I checked them and they are the same)

So I checked some [tutorial](http://google.github.io/proto-lens/installing-protoc.html) on how to install protobuf compiler for ubuntu: 

it says to do this :

```
PROTOC_ZIP=protoc-3.7.1-linux-x86_64.zip
curl -OL https://github.com/protocolbuffers/protobuf/releases/download/v3.7.1/$PROTOC_ZIP
sudo unzip -o $PROTOC_ZIP -d /usr/local bin/protoc
sudo unzip -o $PROTOC_ZIP -d /usr/local 'include/*'
rm -f $PROTOC_ZIP
```





okay so I thought yep, I need to get the zip file.

So I put these in a bash file and put it in the dockerfile:

```
COPY ./get_protobuf.sh .

RUN get_protobuf.sh
```

But wait a minute! why this  is linux and not python?!
okay so I need to get the latest version (3.13 instead of 3.7) but should I go with python or linux version?  
sounds like to me that the python version would need a pip install and then called by python! but anyways I will go on with the exact example..
(looks like I can even install it with just pip install protobuf!)





okay so both of the curl and unzip commands don't work on my container. So either install them or go back to wget and tar.
well since now I now the tar file and zip files are the same,why not!


okay so I need to replace the curl with the wget
so seems like it would be in this shape:
```
PROTOC_TAR=protoc-3.13.1-linux-x86_64.tar.gz
wget https://github.com/protocolbuffers/protobuf/releases/download/v3.13.1/$PROTOC_TAR
```

okay how about the tar one?
```
unzip -o $PROTOC_ZIP -d /usr/local bin/protoc
unzip -o $PROTOC_ZIP -d /usr/local 'include/*'
```
what is this command anyway?

okay so the -o parameters says overwrite without prompt.
and the -d defines the destination 
so I guess with tar in would be :

```
tar -zxvf $PROTOC_TAR  /usr/local bin/protoc
tar -zxvf $PROTOC_TAR /usr/local 'include/*'
```
oh no!!  the release page does not have a tar file for linux, so we get back to zip and change the bash files as:


```
PROTOC_ZIP=protoc-3.13.0-linux-x86_64.zip
wget https://github.com/protocolbuffers/protobuf/releases/download/v3.13.0/$PROTOC_ZIP
unzip -o $PROTOC_ZIP -d /usr/local bin/protoc
unzip -o $PROTOC_ZIP -d /usr/local 'include/*'
rm -f $PROTOC_ZIP
```

and add this to dockeer to install the zip package
```
RUN apt-get install -y zip
```



oaky so seems the protobuf install works, 

### okay what comes after compiler?

next we continue with the example from

https://www.freecodecamp.org/news/googles-protocol-buffers-in-python/

I add the content of the protobuf example into todolist.proto

```


syntax = "proto3";

// Not necessary for Python but should still be declared to avoid name collisions 
// in the Protocol Buffers namespace and non-Python languages
package protoblog;

// Style guide prefers prefixing enum values instead of surrounding
// with an enclosing message
enum TaskState {
    TASK_OPEN = 0;
    TASK_IN_PROGRESS = 1;
    TASK_POST_PONED = 2;
    TASK_CLOSED = 3;
    TASK_DONE = 4;
}

message TodoList {
    int32 owner_id = 1;
    string owner_name = 2;

    message ListItems {
        TaskState state = 1;
        string task = 2;
        string due_date = 3;
    }

    repeated ListItems todos = 3;
}
```

 and then Run the command to complie it 
```
protoc -I=. --python_out=. ./todolist.proto
```

then I add the python main script that import the associated python class for the proto.

```
import todolist_pb2 as TodoList

my_list = TodoList.TodoList()
my_list.owner_id = 1234

# ...

with open("./serializedFile", "wb") as fd:
    fd.write(my_list.SerializeToString())


my_list = TodoList.TodoList()
with open("./serializedFile", "rb") as fd:
    my_list.ParseFromString(fd.read())

print(my_list)
```

okay I name this todolist.py and copy it into the container image.
Then I guess I only need to run this after the container is built.

now my dockerfile looks like this:

```
FROM ubuntu

RUN apt-get update -y 
RUN apt-get install -y git wget build-essential curl

RUN apt-get update && apt-get -y install python3 


RUN apt-get install -y zip
COPY ./get_protobuf.sh .

RUN ./get_protobuf.sh

COPY ./todolist.proto .
COPY ./compile_proto.sh .


RUN ./compile_proto.sh


COPY ./todolist.py .

```



So I get into the conatiner and run the code todolist.py.  

oh shoot, I get some error like this:

```
from google.protobuf.internal import enum_type_wrapper
```

I check the protobuf python class and it look like it is still missing something from the python comile side.

So I think it looks like that the python comipler is missing and I try to install this:

```
pip install protobuf
```

of course the I get the error that pip is not found.

So I need to install pip.  The https://pip.pypa.io/en/stable/installing/
says that we need to do the following:

curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
python get-pip.py

well this means that I need curl, I try with wget first it does not work (?).
I add curl to the first line of the docker build (having both curl and wget is kinda of not necessary though)


Anyway, but still the python get-pip.py does not work. So I look at the linke in the above and it says something in the line of "pip is included with python versions of 3.4 and above. So I think maybe I need to install a higher version of the python I get this for installing the version 3.7
```
cd /
apt update
export DEBIAN_FRONTEND=noninteractive
apt-get install -y --no-install-recommends tzdata
apt install -yq software-properties-common
add-apt-repository -y ppa:deadsnakes/ppa
apt install -y python3.7
```


but still the pip install line does not work ( I need to use pip3 I think)
So I check the web on how to install pip on the docker specifically:

https://stackoverflow.com/questions/36611052/install-pip-in-docker

```
FROM ubuntu:14.04

RUN apt-get update && apt-get install -y \
    software-properties-common
RUN add-apt-repository universe
RUN apt-get update && apt-get install -y \
    apache2 \
    curl \
    git \
    libapache2-mod-php5 \
    php5 \
    php5-mcrypt \
    php5-mysql \
    python3.4 \
    python3-pip
```

okay I think maybe I just try to add the last line to the installation.
and ....  yes it works now!
now the dockerfile looks like this:
```
FROM ubuntu

RUN apt-get update -y 
RUN apt-get install -y git wget build-essential curl

COPY ./install_python.sh .
RUN ./install_python.sh

RUN apt-get install -y zip
COPY ./get_protobuf.sh .

RUN ./get_protobuf.sh

COPY ./todolist.proto .
COPY ./compile_proto.sh .


RUN pip3 install protobuf
RUN ./compile_proto.sh


COPY ./todolist.py .
```

Confession! well if I would have oppend the second link on how to install pip for linux 
https://www.tecmint.com/install-pip-in-linux/

my life for pip and python installation would have been easier!




### Okay now the example works, what is next?

a few questions here:

* how can I remove the unnneded steps of the dockerfile and make it cleaner.

* what if I remove the install part of the protobuf and only install the pip install protobuf.
* what was the second part of the two step installation approach in the google guide? was it the same thing I tried here or something else?

* how can I get into installing and working with the ORtools example.